In [1]:
import os

In [2]:
%pwd

'/home/cdot/PycharmProjects/ChickeDecease/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/cdot/PycharmProjects/ChickeDecease'

In [5]:
import tensorflow as tf

2024-01-02 11:28:16.855420: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-02 11:28:16.898246: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-02 11:28:16.898289: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-02 11:28:16.899597: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-02 11:28:16.905558: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-02 11:28:16.906070: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [6]:
model = tf.keras.models.load_model("artifacts/training/model.h5")


In [7]:
from dataclasses import dataclass
from pathlib import Path

In [8]:
@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int

In [10]:
from CNNClassifier.constants import *
from CNNClassifier.utils.common import read_yaml, create_directories, save_json

In [20]:
class ConfigurationManager:
    def __init__(self, 
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
                 self.config = read_yaml(config_filepath)
                 self.params = read_yaml(params_filepath)
                 create_directories([self.config.artifacts_root])

    def get_validation_config(self) -> EvaluationConfig:
            eval_config = EvaluationConfig(
                    path_of_model="artifacts/training/model.h5",
                    training_data="artifacts/data_ingestion/Chicken-fecal-images/",
                    all_params=self.params,
                    params_image_size=self.params.IMAGE_SIZE,
                    params_batch_size=self.params.BATCH_SIZE)
            return eval_config

In [21]:
from urllib.parse import urlparse

In [22]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):
        datagenerator_kwargs = dict(rescale = 1./255, validation_split = 0.30)
        dataflow_kwargs = dict(target_size = self.config.params_image_size[:-1],
                               batch_size = self.config.params_batch_size,
                               interpolation = "bilinear")
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs)
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = "validation",
            shuffle = False,
            **dataflow_kwargs)
        
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)


    def save_score(self):
        scores = {"loss": self.score[0], "accuracy":self.score[1]}
        save_json(path = Path("scores.json"), data = scores)


In [23]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()
except Exception as e:
    raise e

[2024-01-02 12:06:35,768: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-01-02 12:06:35,776: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-02 12:06:35,779: INFO: common: Created directory at: artifacts]


Found 116 images belonging to 2 classes.


2024-01-02 12:06:36.397941: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205520896 exceeds 10% of free system memory.
2024-01-02 12:06:36.473508: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205520896 exceeds 10% of free system memory.
2024-01-02 12:06:37.279796: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 102760448 exceeds 10% of free system memory.
2024-01-02 12:06:37.385519: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 102760448 exceeds 10% of free system memory.


1/8 [==>...........................] - ETA: 17s - loss: 0.0000e+00 - accuracy: 1.0000

2024-01-02 12:06:38.690824: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205520896 exceeds 10% of free system memory.


8/8 [==============================] - 13s 1s/step - loss: 8.5575 - accuracy: 0.5000
[2024-01-02 12:06:48,811: INFO: common: Now JSON file format saved at: scores.json]
